In [1]:
from selenium import webdriver
import os
from SICOIN_scraper import *
import xlrd
import re
import pandas as pd
import time

In [3]:
options = webdriver.ChromeOptions()
options.headless = False # True
# options.add_argument("download.default_directory="+os.getcwd())
options.add_experimental_option("prefs", {
  "download.default_directory": os.path.abspath("../../../Resource Tracking/SICOIN GT/SICOIN - SIGES - Trimestral"),
  "download.prompt_for_download": False,
})
driver = webdriver.Chrome(chrome_options=options, executable_path="../../../../Downloads/chromedriver_win32 77/chromedriver.exe")
driver.get('https://sicoin.minfin.gob.gt')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [4]:
login(driver)
#driver.close()

<a href="#Datos-por-trimestre">Ir a datos por trimestre</a>
# Anual

In [70]:
# IGRC01 - Ejecución Financiera
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh

In [ ]:
# IGRC03 - Comparacion de Ejecucion Financiera y Fisica
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [69]:
# IGRC02 - Ejecución Física
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [71]:
# IGRC00, IGRC01, IGRC02
# Malaria
for year in range(2013, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [ ]:
# IGRC00, IGRC01, IGRC02
# HIV
for year in range(2013, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

# Preprocessing:

In [2]:
df = SICOIN_extractTable(s, igrc2_options)
df

In [123]:
def nextCell(row, offset = 0):
    lenr = len(row)
    if offset >= lenr:
        return None
    while (offset+1 < lenr) & (row[offset].ctype == xlrd.XL_CELL_EMPTY):
        offset += 1
    return offset

def process_row(row, truncate = None):
    rowData = []
    js = []
    j = nextCell(row, 0)
    while True:
        if j is not None:
            if (truncate is not None) and (truncate < j):
                break
            value = str(row[j].value).strip()
            js.append(j)
            rowData.append(value)
        if j is None:
            break    
        j = nextCell(row, j+1)
        
    return rowData, js

def tryFun(fun, param1):
    try:
        return fun(param1)
    except:
        return None

def process_table_simple(sheet, options):
    i = options["start"]
    df = pd.DataFrame()
    while True:
        processTitleRow = False
        popCategory = False
        i += 1
        if i >= sheet.nrows:
            break
        row = sheet.row(i)
        values, js = process_row(row, options["truncate"] if "truncate" in options else None)
        
        if len(values)==0:
            continue
        if values[0].lower().startswith("tota"):
            continue
        rowData = pd.Series(values, js)
        df = df.append(rowData, ignore_index=True)
    return df

def process_igrc3(s):
    df = process_table_simple(s, {"start": 22})
    df = df.sort_index(1)
    df.columns = ["CodPrograma", "Programa", "CodAct", "ConceptoID", "CodOb", "Concepto", 
                  "SNIP", "Unidad", "Inicial", "Vigente", "Ejecutado", "PorcentajeE", 
                  "PresupuestoInicial",
                  "PresupuestoVigente", "PresupuestoEjecutado", "PresupuestoPorcE"]
    for col in ["CodPrograma", "Programa", "CodAct", 
                "ConceptoID", "CodOb"]:
        df[col] = df[col].fillna(method="ffill")
    df.loc[df["ConceptoID"].isna(), "ConceptoID"] = "-"
    df = df.loc[df["Concepto"].isna()==False, ]
    return df


def process_igrc1(s):
    df = process_table_simple(s, {"start": 20, "truncate": 25})
    df = df.sort_index(1)
    df.columns = ["Programa", "Subprograma", "Proyecto", "ActOb", "CodRenglon", "Renglon",
                  "Asignado", "Modificado", "Vigente", "Ejecutado", "PorcentajeE"]
    for col in ["Programa", "Subprograma", "Proyecto", "ActOb"]:
        df[col] = df[col].fillna(method="ffill")
    df = df.loc[df["Renglon"].isna()==False, ]
    return df

def process_igrc2(s):
    df = process_table_simple(s, {"start": 22, "truncate": 30})
    df = df.sort_index(1)
    df.loc[df[1].notna(),1] = df[1] + "-" + df[8]
    df.loc[df[2].notna(),2] = df[2] + "-" + df[8]
    df.loc[df[3].notna(),3] = df[3] + "-" + df[8]
    df.loc[df[4].notna(),4] = df[4] + "-" + df[9]
    df.loc[df[6].notna(),6] = df[6] + "-" + df[10]
    del df[8]
    del df[21]
    df.columns = ["Programa", "Subprograma", "Proyecto", "ActOb", "Resultado", "LugarCod", "LocationNombre", 
                  "Unidades", "Inicial", "Vigente", "LugarEjecutado", "Ejecutado", "PorcEj", "NBeneficiados"]
    for col in ["Programa", "Subprograma", "Proyecto", "ActOb", "Resultado", "Unidades"]:
        df[col] = df[col].fillna(method="ffill")
    df = df.loc[df["Resultado"].isna()==False, ]
    return df

In [4]:
# Read all excel files:
xls_files = []
for root, dirs, files in os.walk("../../../Resource Tracking/SICOIN GT/SICOIN - Gestion"):
    n = len(files)
    for file in files:
        if (file.endswith(".xls") or file.endswith(".xlsx")):
            print(file)
            xls_files.append(xlrd.open_workbook(root + "/" + file))

In [6]:
igrc01df = pd.DataFrame()
igrc02df = pd.DataFrame()
igrc03df = pd.DataFrame()

for i, x in enumerate(xls_files):
    try: 
        s = x.sheet_by_index(0)
        year =  0
        program = None
        dataset = None
        if str(s.row(3)[2].value).endswith("IGRC01"):
            year = s.row(15)[0].value.strip()[-4:]
            dataset = "EjecucionFin"
            program = s.row(13)[2].value
            temp = process_igrc1(s)
            temp["Year"] = year
            temp["Reporte_Programa"] = program
            igrc01df = igrc01df.append(temp, ignore_index = True)
        elif str(s.row(1)[5].value).endswith("IGRC02"):
            year = s.row(15)[8].value
            dataset = "EjecucionFisica"
            program = s.row(8)[5].value
            temp = process_igrc2(s)
            temp["Year"] = year
            temp["Reporte_Programa"] = program
            igrc02df = igrc02df.append(temp, ignore_index = True)
        elif str(s.row(1)[6].value).endswith("IGRC03"):
            year = s.row(15)[10].value
            dataset = "EjecucionFinFis"
            program = s.row(8)[6].value
            temp = process_igrc3(s)
            temp["Year"] = year
            temp["Reporte_Programa"] = program
            igrc03df = igrc03df.append(temp, ignore_index = True)
        else:
            print("Unknown type of report")
        if program:
            program_pre = re.search("entidad (\w* )?\= (\d*)\, programa \= (\d*)", 
                                                    program, re.IGNORECASE)
            if program_pre is not None:
                program = "-".join(program_pre.groups()[1:3])
        print(dataset, year, program)
    except:
        print("Bad report: i=", i)

EjecucionFinFis 2018 11130009-16
EjecucionFisica 2017 11130009-16
EjecucionFinFis 2017 11130009-16
EjecucionFinFis 2017 11130009-18
EjecucionFisica 2015 11130009-17
EjecucionFin 2014 11130009-17
EjecucionFinFis 2014 11130009-18
EjecucionFin 2014 11130009-16
EjecucionFin 2015 11130009-18
EjecucionFisica 2017 11130009-18
Bad report: i= 10
EjecucionFisica 2018 11130009-18
EjecucionFinFis 2014 11130009-16
EjecucionFisica 2018 11130009-16
EjecucionFinFis 2015 11130009-16
EjecucionFin 2016 11130009-18
EjecucionFisica 2018 11130009-17
EjecucionFinFis 2018 11130009-18
EjecucionFin 2016 11130009-17
EjecucionFinFis 2017 11130009-17
EjecucionFisica 2017 11130009-17
EjecucionFin 2015 11130009-17
EjecucionFinFis 2013 11130009-16
EjecucionFin 2017 11130009-16
EjecucionFisica 2014 11130009-18
EjecucionFinFis 2016 11130009-18
EjecucionFin 2016 11130009-16
EjecucionFisica 2014 11130009-16
EjecucionFisica 2013 11130009-18
EjecucionFinFis 2013 11130009-18
EjecucionFisica 2015 11130009-18
EjecucionFinFis 

In [7]:
badrep = xls_files[10].sheet_by_index(0)

In [17]:
for i in range(0,29):
    print("-".join([str(x.value) for x in badrep.row(i)]))

---------------------------
--Sistema de Contabilidad Integrada Gubernamental------------------PAGINA   :--1.0---DE--1.0
---------------------------
--Informes de Gestion y Rendicion de Cuentas  - IGRC01------------------FECHA     :---43510.0----
---------------------------
---------------------------
--Ejecución Financiera------------------HORA       :---0.48822916666666666----
---------------------------
---------------------------
--Expresado en Quetzales------------------REPORTE:---R00820850.rpt----
---------------------------
--DEL MES DE ENERO AL MES DE DICIEMBRE-------------------------
---------------------------
--Entidad Institucional = 11130009, Programa = 17-------------------------
---------------------------
EJERCICIO: 2012        --------------CORTE: ENTIDAD-------------
---------------------------
PG - SPG - PY - ACT- OB - REN-----------ASIGNADO--MODIFICADO---VIGENTE--DEVENGADO------- % EJEC --
---------------------------
ENTIDAD:    ---------------------------
--------

In [18]:
# Bad report was actually empty, so its ok

In [20]:
igrc01df.columns

Index(['Programa', 'Subprograma', 'Proyecto', 'ActOb', 'CodRenglon', 'Renglon',
       'Asignado', 'Modificado', 'Vigente', 'Ejecutado', 'PorcentajeE', 'Year',
       'Reporte_Programa'],
      dtype='object')

In [22]:
igrc01df.to_csv("../../../Resource Tracking/SICOIN GT/IGRC01_PCE_Ejecucion_Financiera.csv")

In [27]:
igrc02df.to_csv("../../../Resource Tracking/SICOIN GT/IGRC02_PCE_Ejecucion_Fisica.csv")

In [26]:
igrc03df.to_csv("../../../Resource Tracking/SICOIN GT/IGRC03_PCE_EjecucionFisicaFinanciera.csv")

In [30]:
igrc02df["LugarEjecutado"] = igrc02df.LugarEjecutado.astype(float)

In [34]:
igrc02df.Resultado.unique()

array(['1.0-Persona atendida para la prevención de ITS, VIH/SIDA',
       '3.0-Persona de mayor vulnerabilidad (PEMARV) que recibe condones (incluye consejería)',
       '4.0-Mujer VIH positiva que recibe consejería sistemática post parto VIH positiva para efectuar seguimiento al niño expuesto',
       '5.0-Persona adolescente, adulto, adulto mayor y mujer embarazada, capacitada sobre formas de prevención de las ITS, VIH/SIDA  y el uso correcto del condón',
       '6.0-Apoyo en la gestión administrativa en la prevención de ITS, VIH/SIDA',
       '1.0-Persona con diagnóstico y tratamiento de ITS, VIH/SIDA',
       '2.0-Persona/caso sospechoso de VIH con prueba diagnóstica, incluye orientación pre y post prueba',
       '3.0-Mujer embarazada con diagnóstico de VIH',
       '4.0-Mujer embarazada referida para tratamiento ARV y/o resolución del embarazo por cesárea (prevención de la transmisión vertical)',
       '5.0-Neonato, hijo de madre VIH positiva con tratamiento profiláctico',
     

In [48]:
igrc02df[igrc02df.LugarCod.notna() & 
         (igrc02df.Resultado.isin([
'7.0-Persona beneficiada con uso de pabellón impregnado con insecticida de larga duración para el control del vector de la malaria en su fase adulta',
'7.0-Persona beneficiada con uso de pabellón impregnado con insecticida de larga duración para el control del vector de la malaria en su fase adulta como parte del manejo integrado de vectores',
'7.0-Control  del Vector con uso de pabellones impregnados con Deltametrina.',
'8.0-Pruebas Biologicas sobre pabellones impregnados con Deltamentrina.'
]))]\
    .groupby(["Year", "LugarCod"])\
    .LugarEjecutado.sum().unstack(0).fillna(0).sum()

Year
2013    118034.0
2014     27812.0
2015     47481.0
2016    335705.0
2017    178233.0
2018    136258.0
dtype: float64

In [45]:
igrc02df[igrc02df.LugarCod.notna() & 
         (igrc02df.Resultado.isin([
'8.0-Persona con diagnóstico de tuberculosis en condición de egreso curado',
'8.0-Casos de Tuberculosis en condición de egreso curado'
]))]\
    .groupby(["Year", "LugarCod"])\
    .LugarEjecutado.sum().unstack(0).fillna(0)

Year,2013,2014,2015,2016,2017,2018
LugarCod,,,,,,
0101,183.0,68.0,129.0,143.0,122.0,101.0
0102,0.0,0.0,1.0,2.0,4.0,1.0
0103,1.0,0.0,5.0,3.0,6.0,0.0
0104,0.0,0.0,1.0,0.0,1.0,0.0
0105,0.0,0.0,6.0,4.0,8.0,2.0
0106,1.0,0.0,6.0,8.0,12.0,1.0
0107,0.0,0.0,6.0,4.0,0.0,1.0
0108,21.0,16.0,0.0,0.0,0.0,0.0
0109,2.0,1.0,0.0,0.0,0.0,0.0


In [46]:
igrc01df.head()

,Programa,Subprograma,Proyecto,ActOb,CodRenglon,Renglon,Asignado,Modificado,Vigente,Ejecutado,PorcentajeE,Year,Reporte_Programa
0,17 PREVENCIÓN Y CONTROL DE LA TUBERCULOSIS,00 SIN SUBPROGRAMA,000 SIN PROYECTO,001 000 PREVENCIÓN DE LA TUBERCULOSIS,011,PERSONAL PERMANENTE,438744.0,14304.0,453048.0,443987.3,98.00005738906253,2014,"Entidad Institucional = 11130009, Programa = 17"
1,17 PREVENCIÓN Y CONTROL DE LA TUBERCULOSIS,00 SIN SUBPROGRAMA,000 SIN PROYECTO,001 000 PREVENCIÓN DE LA TUBERCULOSIS,012,COMPLEMENTO PERSONAL AL SALARIO DEL PERSONAL P...,300.0,0.0,300.0,300.0,100.0,2014,"Entidad Institucional = 11130009, Programa = 17"
2,17 PREVENCIÓN Y CONTROL DE LA TUBERCULOSIS,00 SIN SUBPROGRAMA,000 SIN PROYECTO,001 000 PREVENCIÓN DE LA TUBERCULOSIS,013,COMPLEMENTO POR ANTIGÜEDAD AL PERSONAL PERMANENTE,6240.0,30.0,6270.0,6255.0,99.76076555023923,2014,"Entidad Institucional = 11130009, Programa = 17"
3,17 PREVENCIÓN Y CONTROL DE LA TUBERCULOSIS,00 SIN SUBPROGRAMA,000 SIN PROYECTO,001 000 PREVENCIÓN DE LA TUBERCULOSIS,014,COMPLEMENTO POR CALIDAD PROFESIONAL AL PERSONA...,22500.0,0.0,22500.0,21662.5,96.27777777777777,2014,"Entidad Institucional = 11130009, Programa = 17"
4,17 PREVENCIÓN Y CONTROL DE LA TUBERCULOSIS,00 SIN SUBPROGRAMA,000 SIN PROYECTO,001 000 PREVENCIÓN DE LA TUBERCULOSIS,015,COMPLEMENTOS ESPECIFICOS AL PERSONAL PERMANENTE,774078.0,604104.0,1378182.0,999297.63,72.50839366643885,2014,"Entidad Institucional = 11130009, Programa = 17"


In [47]:
igrc03df.head()

,CodPrograma,Programa,CodAct,ConceptoID,CodOb,Concepto,SNIP,Unidad,Inicial,Vigente,Ejecutado,PorcentajeE,PresupuestoInicial,PresupuestoVigente,PresupuestoEjecutado,PresupuestoPorcE,Year,Reporte_Programa
0,16,"PREVENCIÓN Y CONTROL DE ITS, VIH/SIDA",001,-,000,"PERSONA ATENDIDA PARA LA PREVENCIÓN DE ITS, VI...",NaN,NaN,NaN,NaN,NaN,NaN,67080922.0,63405499.0,48718840.08,76.84,2018,"ENTIDAD = 11130009, PROGRAMA = 16"
1,16,"PREVENCIÓN Y CONTROL DE ITS, VIH/SIDA",001,1.0,000,"Persona atendida para la prevención de ITS, VI...",0.0,PERSONA,1553045.0,1279704.0,739254.0,57.77,NaN,NaN,NaN,,2018,"ENTIDAD = 11130009, PROGRAMA = 16"
2,16,"PREVENCIÓN Y CONTROL DE ITS, VIH/SIDA",001,3.0,000,Persona de mayor vulnerabilidad (PEMARV) que r...,0.0,PERSONA,264099.0,202884.0,123763.0,61.0,NaN,NaN,NaN,,2018,"ENTIDAD = 11130009, PROGRAMA = 16"
3,16,"PREVENCIÓN Y CONTROL DE ITS, VIH/SIDA",001,4.0,000,Mujer VIH positiva que recibe consejería siste...,0.0,PERSONA,30968.0,14136.0,11236.0,79.49,NaN,NaN,NaN,,2018,"ENTIDAD = 11130009, PROGRAMA = 16"
4,16,"PREVENCIÓN Y CONTROL DE ITS, VIH/SIDA",001,5.0,000,"Persona adolescente, adulto, adulto mayor y mu...",0.0,PERSONA,1439691.0,1263161.0,735374.0,58.22,NaN,NaN,NaN,,2018,"ENTIDAD = 11130009, PROGRAMA = 16"


# Datos por trimestre

In [5]:
# Reports following a typical pattern found in SICOIN, where filters are set in an upper frame
# while month option is put in the lower frame and then a form is submitted to download an excel file
def reportTypeA(driver, clicks_sequence, filters, mes_inicio = "ENERO", mes_final = "DICIEMBRE"):
    getReportGUI(driver, clicks_sequence)
    time.sleep(3)
    findHeading(driver)
    pags = driver.find_element_by_id("paginas")
    driver.switch_to.frame(pags)
    frames = driver.find_elements_by_tag_name("frame")
    while (len(frames) == 0):
        print("waiting for page load...")
        time.sleep(10)
        driver.switch_to.parent_frame()
        pags = driver.find_element_by_id("paginas")
        driver.switch_to.frame(pags)
        frames = driver.find_elements_by_tag_name("frame")

    driver.switch_to.frame(frames[0])
    for filter in filters:
        variable = driver.find_element_by_id("UCFiltroReporte1_DDLEstructura")
        operador = driver.find_element_by_id("UCFiltroReporte1_DDLOperador")
        valor = driver.find_element_by_id("UCFiltroReporte1_TXTValor")
        variable.send_keys(filter["variable"])
        operador.send_keys(filter["operador"])
        valor.send_keys(filter["valor"])
        driver.find_element_by_id("UCFiltroReporte1_Button1").click()
        time.sleep(2)
    driver.switch_to.parent_frame()
    driver.switch_to.frame(frames[1])
    driver.find_element_by_id("P_MES_INI").send_keys(mes_inicio)
    driver.find_element_by_id("P_MES_FIN").send_keys(mes_final)
    driver.find_elements_by_name("opExport")[1].click()
    driver.find_element_by_id("sbtContinuar").click()
    driver.refresh()

def setYear(driver, year):
    driver.switch_to.parent_frame()
    frames = driver.find_elements_by_tag_name("frame")
    driver.switch_to.frame(frames[0])
    try:
        preselector = driver.find_element_by_id("lblEjercicioActual")
        preselector.click()
    except:
        pass
    selector = driver.find_element_by_id("ddlListaEjercicios")
    try:
        ix = [x[-4:] for x in selector.get_attribute("innerHTML").split("</option>")].index(str(year))
        
    except:
        print("Year not found " + year)
        return False
    
    selector.click()
    selector.send_keys(str(year))
    selector.send_keys(webdriver.common.keys.Keys.ENTER)
    driver.switch_to.parent_frame()
    return True

In [12]:
pags = driver.find_element_by_id("paginas")

In [11]:
driver.switch_to.parent_frame()

In [201]:
driver.switch_to.parent_frame()
driver.switch_to.frame(frames[1])    

In [202]:
driver.find_element_by_id("P_MES_INI")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="P_MES_INI"]"}
  (Session info: chrome=77.0.3865.90)


In [6]:
trimestres = [
    ("ENERO", "MARZO"), 
    ("ABRIL", "JUNIO"), 
    ("JULIO", "SEPTIEMBRE"), 
    ("OCTUBRE", "DICIEMBRE")
]

In [7]:
driver.refresh()
setYear(driver, year)

NameError: name 'year' is not defined

In [13]:
# IGRC01 - Ejecución Financiera
# TB
for year in range(2013, 2018):
    driver.refresh()
    setYear(driver, year)
    print(year)
    for m1, m2 in trimestres:
        print(m1,m2)
        reportTypeA(driver,
            clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
            filters = [{
                    "variable": "ENTIDAD",
                    "operador": "igual",
                    "valor"   : "11130009"
                },
                {
                    "variable": "PROGRAMA",
                    "operador": "igual",
                    "valor"   : "17"
                }
            ], mes_inicio = m1, mes_final = m2)
        # The frames get all messed up, so we simply refresh the ugly webpage
        driver.refresh()


2013
ENERO MARZO
ABRIL JUNIO


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="UCFiltroReporte1_DDLEstructura"]"}
  (Session info: chrome=77.0.3865.120)


In [12]:

# IGRC03 - Comparacion de Ejecucion Financiera y Fisica
# TB
for year in range(2013, 2018):
    setYear(driver, year)
    print(year)
    for m1, m2 in trimestres:
        print(m1,m2)
        reportTypeA(driver,
            clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
            filters = [{
                    "variable": "ENTIDAD",
                    "operador": "igual",
                    "valor"   : "11130009"
                },
                {
                    "variable": "PROGRAMA",
                    "operador": "igual",
                    "valor"   : "17"
                }
            ], mes_inicio = m1, mes_final = m2)
        # The frames get all messed up, so we simply refresh the ugly webpage
        driver.refresh()

# IGRC02 - Ejecución Física
# TB
for year in range(2013, 2018):
    setYear(driver, year)
    print(year)
    for m1, m2 in trimestres:
        print(m1,m2)
        reportTypeA(driver,
            clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
            filters = [{
                    "variable": "ENTIDAD",
                    "operador": "igual",
                    "valor"   : "11130009"
                },
                {
                    "variable": "PROGRAMA",
                    "operador": "igual",
                    "valor"   : "17"
                }
            ], mes_inicio = m1, mes_final = m2)
        # The frames get all messed up, so we simply refresh the ugly webpage
        driver.refresh()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="ddlListaEjercicios"]"}
  (Session info: chrome=77.0.3865.90)
